In [91]:
import pandas as pd
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import MinMaxScaler, StandardScaler, normalize

In [101]:
initial_dataset = pd.read_csv("biodegradable_a.csv")
total_len, _ = initial_dataset.shape

# NOTE - NO INDEPENDENT VALIDATION SET !!!

# Total with means
#categorical = ['int16', 'int32', 'int64']
#biodegradable = ['object']
#numerical = ['float16', 'float32', 'float64']

class_cols = [col for col in initial_dataset.drop("Biodegradable", axis=1) if initial_dataset[col].apply(lambda x: x % 1 == 0).all()]
num_cols = [col for col in initial_dataset.drop("Biodegradable", axis=1) if initial_dataset[col].apply(lambda x: x % 1 != 0).all()]

print(len(class_cols) + len(num_cols))
print(len(initial_dataset.drop("Biodegradable", axis=1).columns))

total_categorical_dataset = initial_dataset[class_cols]
#total_categorical_dataset = total_categorical_dataset.fillna(total_categorical_dataset.mode())
total_categorical_dataset = total_categorical_dataset.fillna(-1)

total_numerical_dataset = initial_dataset[num_cols]
total_numerical_dataset = total_numerical_dataset.fillna(total_numerical_dataset.mean())

total_biodegradable = initial_dataset["Biodegradable"]
#total_biodegradable = initial_dataset.select_dtypes(include=biodegradable)
total_biodegradable = total_biodegradable.fillna("")

# Scale numerical data
# https://scikit-learn.org/stable/modules/preproce
print(total_numerical_dataset)

scaler = StandardScaler()
#scaler = MinMaxScaler(feature_range=(-1, 1))

total_numerical_dataset = pd.DataFrame(scaler.fit_transform(total_numerical_dataset),
             columns=total_numerical_dataset.columns, index=total_numerical_dataset.index)


#total_numerical_dataset = pd.DataFrame(normalize(total_numerical_dataset, norm='l2', axis=1, copy=True, return_norm=False),
#             columns=total_numerical_dataset.columns, index=total_numerical_dataset.index)

print(total_numerical_dataset)
#
total_dataset = pd.concat([total_categorical_dataset, total_numerical_dataset,total_biodegradable], axis=1)

total_len, _ = total_dataset.shape
train_dataset_len = round(total_len * 0.75)

dataset_train = total_dataset[0:train_dataset_len]
dataset_test = total_dataset[train_dataset_len:total_len]

print(total_dataset.shape)

# Removal of None/NaN vals
dropna_dataset = initial_dataset.dropna()

dropna_len, _ = dropna_dataset.shape
model_dropna_len = round(total_len * 0.75)

dropna_train = dropna_dataset[0:model_dropna_len]
dropna_test = dropna_dataset[model_dropna_len:dropna_len]

print(dropna_dataset.shape)

21
41
         SM6_L        Mi  SpPosA_B
0     9.002000  1.142000  1.201000
1     8.723000  1.144000  1.104000
2     9.110000  1.152000  1.092000
3     6.594000  1.167000  1.024000
4     9.528000  1.147000  1.137000
...        ...       ...       ...
4559  8.617289  1.150358  1.119962
4560  8.099587  1.139562  1.279367
4561  9.225643  1.148667  1.126555
4562  9.521317  1.142303  1.150385
4563  8.499379  1.162866  1.109430

[4564 rows x 3 columns]
         SM6_L        Mi  SpPosA_B
0    -0.710938  0.328503  0.000615
1    -1.039247  0.423761 -1.207722
2    -0.583850  0.804797 -1.357207
3    -3.544517  1.519239 -2.204289
4    -0.091975  0.566650 -0.796638
...        ...       ...       ...
4559 -1.163640  0.726586 -1.008886
4560 -1.772839  0.212358  0.976838
4561 -0.447769  0.646039 -0.926756
4562 -0.099839  0.342948 -0.629899
4563 -1.302390  1.322359 -1.140085

[4564 rows x 3 columns]
(4564, 22)
(889, 42)


In [93]:
print(total_dataset.columns)
print(dropna_test.columns)

Index(['nHM', 'F04', 'NssssC', 'nCb', 'nO', 'F03', 'nN_N', 'nArNO2', 'nCRX3',
       'B01', 'B03', 'N_073', 'B04', 'C_026', 'F02_CN', 'nHDon', 'nN',
       'nArCOOR', 'SM6_L', 'Mi', 'SpPosA_B', 'Biodegradable'],
      dtype='object')
Index(['SpMax_L', 'J_Dz(e)', 'nHM', 'F01', 'F04', 'NssssC', 'nCb', 'C', 'nCp',
       'nO', 'F03', 'SdssC', 'HyWi_B', 'LOC', 'SM6_L', 'F03_CO', 'Me', 'Mi',
       'nN_N', 'nArNO2', 'nCRX3', 'SpPosA_B', 'nCIR', 'B01', 'B03', 'N_073',
       'SpMax_A', 'Psi_i_1d', 'B04', 'SdO', 'TI2_L', 'nCrt', 'C_026', 'F02_CN',
       'nHDon', 'SpMax_B', 'Psi_i_A', 'nN', 'SM6_B', 'nArCOOR', 'nX',
       'Biodegradable'],
      dtype='object')


## Using Model with replaced values when NaN, and discarding the dropped NaN values dataset

In [94]:
X_train = dataset_train.drop(["Biodegradable"], axis=1)
y_train = dataset_train.Biodegradable
print(X_train)
print(y_train)

      nHM  F04  NssssC  nCb   nO  F03  nN_N  nArNO2  nCRX3  B01  ...  N_073  \
0     0.0  0.0     0.0  0.0  0.0  0.0   0.0     0.0    0.0  0.0  ...    0.0   
1     0.0  0.0     0.0  0.0  1.0  0.0   0.0     0.0    0.0  0.0  ...    0.0   
2     0.0  0.0     0.0  0.0  4.0  0.0   0.0     0.0    0.0  0.0  ...    0.0   
3     0.0  0.0     0.0  0.0  2.0  0.0   0.0     0.0    0.0  0.0  ...    0.0   
4     0.0  0.0     0.0  0.0  4.0  0.0   0.0     0.0    0.0  0.0  ...    0.0   
...   ...  ...     ...  ...  ...  ...   ...     ...    ...  ...  ...    ...   
3418  0.0  0.0     0.0  2.0  2.0  0.0   0.0     0.0    0.0  0.0  ...    0.0   
3419  0.0  0.0     0.0  3.0  3.0  2.0   0.0     0.0    0.0  0.0  ...    0.0   
3420  0.0  0.0     0.0  2.0  2.0  0.0   0.0     0.0    0.0  0.0  ...    0.0   
3421  0.0  0.0     0.0  0.0  2.0  0.0   0.0     0.0    0.0  0.0  ...    0.0   
3422  0.0  0.0     0.0  0.0  1.0  0.0   0.0     0.0    0.0  0.0  ...    0.0   

      B04  C_026  F02_CN  nHDon   nN  nArCOOR     S

In [95]:
X_test = dataset_test.drop(["Biodegradable"], axis=1)
y_test = dataset_test.Biodegradable

## Testing Ridge with a Linear Model

In [96]:
#ridge = Ridge(alpha=10, max_iter=9999999).fit(X_train, y_train)

#print("The bias is: ",  ridge.intercept_)
#print("The other parameters are: ")
#for i, beta in enumerate(ridge.coef_):
#    print("\t B%02d -> %9.3f"% (i+1, beta))

## Testing lasso with a Linear Model

In [97]:
#L = Lasso(alpha=2, max_iter=9999999).fit(X_train, y_train)

#print("The bias is: ",  L.intercept_)
#print("The other parameters are: ")
#for i, beta in enumerate(L.coef_):
#    print("\t B%02d -> %9.3f"% (i+1, beta))
#preds=L.predict(X_test)
#printRegStatistics(y_test, preds)